In [16]:
import pyvista
import lfricflux
import numpy
import mint
import vtk

In [17]:
inputFile = '../data/gungho/original/lfric_diag.nc'
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [18]:
# create an empty vector field object
x, y = lf.getEdgeLonLat()
vf = lfricflux.VtkVectors(x, y, vector_field_name='velocity', cartesian=True)

In [19]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [20]:
# last dimension is number of edges
u.shape

(1, 30, 3072)

In [21]:
# set the vector field
vf.setField(u, v)
# set time = 0, elev = 0
vf.setSlice((0, 0))

In [22]:
# vector field
arrow = vtk.vtkArrowSource()
glyphs = vtk.vtkGlyph3D()
glyphs.SetScaleModeToScaleByVector()
glyphs.SetColorModeToColorByVector()
glyphs.SetInputData(vf.getMesh())
glyphs.SetSourceConnection(arrow.GetOutputPort())

In [25]:
p = pyvista.Plotter(window_size=(900, 600))
p.add_mesh(glyphs)
p.show()

Widget(value="<iframe src='http://localhost:55985/index.html?ui=P_0x17fbcdea0_3&reconnect=auto' style='width: …

In [26]:
from ipywidgets import interact
import ipywidgets as widgets

def callback(k):
    # copy the new values
    vf.setSlice((0, k))
    print(f'k = {k}')
    vf.vectorField.Modified()

interact(callback, k=widgets.IntSlider(min=0, max=u.shape[1] - 1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='k', max=29), Output()), _dom_classes=('widget-interact',…

<function __main__.callback(k)>